## Develop Pipeline for monthly processing

For month_nn_, where _nn_=1..72. Steps are:
1. chdir to, creating if necessary, a folder month_nn_ under folder P301\_monthly, which contains the basic config.txt file. 
   
2. run the stage "monthly": this creates the model from the 6-year version, fixing the spectral shape for all the 6-year sources, then runs the stage "addseeds\_pgw".
 * initializes process.Process with the flag fix_spectrum_flag=True. All sources that have TS>5 are saved with a flag fixed_spectrum set, which should prevent the sources from being refit, except for normalization, or moved.

  * need to start update_seeds, but currently need the plots seedcheck.

3. addseeds_pgw will incorporate successful seeds -- this triggers a full iteration cycle ending with "finish_month"

4. run the stage "tables". This makes nside=512 TS and kde tables for the sky. The "check_jobs" process runs a clustering analysis for the part of the sky not masked by the path of the Sun, creating a file seeds_ts.txt. The stage is followed by a "addseeds_ts". This adds the successful seeds from the TS map analysis. also triggers an iteration cycle, ending with "finish_month"



In [7]:
%matplotlib inline
month=6
path=os.path.expandvars('$FERMI/skymodels/P301_monthly/month%02d'%month)
if not os.path.exists(path): 
    os.makedirs(path)
    print 'created %s' %path
os.chdir(path)
from uw.like2 import process; reload(process)
map(reload, (process,));

## Document the relevant stages
A dict in pipeline.stagdict defines behavior

In [8]:
from uw.like2.pipeline import stagedict; reload(stagedict);
stagenames =stagedict.stagenames
t=pd.DataFrame(stagenames)
t['monthly addseeds_pgw  finish_month tables addseeds_ts'.split()].T['next sum help'.split()]

,next,sum,help
monthly,addseeds_pgw,menu config counts,create a monthly model; followed by adding PGW...
addseeds_pgw,update_full,config counts pgwave,start update sequence with PGW seeds
finish_month,None,transientinfo,plots for monthly transients
tables,addseeds_ts,hptables,Create tsmap and kde maps
addseeds_ts,update_full,config counts,start update sequence with TS maap seeds


### Test code for creating a new monthly model

In [9]:
r = process.Process('.', fix_spectra_flag=True, quiet=True)

invoking Data.setPhotonBinner for front/back event types...
Failed gti check:
  ltcube: Gti: 462 intervals from 252480000 to 255110000, on time 2192898 
 binfile: Gti: 469 intervals from 252480028 to 255109993, on time 2192653


In [10]:
r.setup_roi(1154)

17 total sources: 0 extended, 3 global


In [11]:
r.process()

2015-08-21 13:56:22 - HP12_1154 - interactive
------------------------------------------------------------------------------------------ 
	 Nearby sources within 5.0 degrees before fit, logL=-4216
name               dist        ra       dec        TS eflux(eV)    index   energy   beta/b
P86Y1143            3.2    77.358     5.700        20      12.1*    2.09      954     0
P86Y1203            4.6    81.471    -0.876                10.7     2.10     2164     0
------------------------------------------------------------------------------------------ 
	Diffuse sources
------------------------------------------------------------------------------------------
ring                        1.00       0.00 
isotrop                     1.00 
SunMoon                     1.00 
------------------------------------------------------------------------------------------
10 calls, function value, improvement, quality: -4216.4, 0.00, 0.00
Name                  index     value  error(%)  gradient
P86Y11

## Test code for checking the PGWAVE seeds
This is against the current sky model. This ROI was just fit above, adjusting the 6-year spectral normalizations in the ROI. (Adjacent ROI's were not adjusted in this test.)

In [12]:
from uw.like2 import (process,seeds,); map(reload,(process,seeds,))
r = process.Process('.', seed_key='pgw', quiet=True)

invoking Data.setPhotonBinner for front/back event types...
Failed gti check:
  ltcube: Gti: 462 intervals from 252480000 to 255110000, on time 2192898 
 binfile: Gti: 469 intervals from 252480028 to 255109993, on time 2192653


In [ ]:
r.setup_roi(840)

77 total sources: 3 extended, 3 global


/afs/slac/g/glast/groups/catalog/python/anaconda/lib/python2.7/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)


In [ ]:
r.process()

2015-08-21 13:57:08 - HP12_0840 - interactive
added PGW_06_104_04 at SkyDir(270.192,-23.594)
added PGW_06_104_08 at SkyDir(269.456,-23.783)
added PGW_06_104_09 at SkyDir(268.331,-26.054)
added PGW_06_104_10 at SkyDir(266.470,-28.826)
added PGW_06_104_14 at SkyDir(267.055,-27.977)
added PGW_06_072_02 at SkyDir(270.261,-23.635)
added PGW_06_072_03 at SkyDir(268.221,-26.284)
added PGW_06_072_04 at SkyDir(266.486,-28.835)
added PGW_06_088_07 at SkyDir(270.140,-23.507)
added PGW_06_088_11 at SkyDir(269.396,-23.691)
added PGW_06_088_12 at SkyDir(268.205,-26.523)
added PGW_06_088_16 at SkyDir(268.627,-25.879)
added PGW_06_088_17 at SkyDir(269.184,-25.019)
added PGW_06_088_18 at SkyDir(267.779,-27.164)
Minimizer warning: bad errors for values ['PGW_06_104_04_Norm' 'PGW_06_104_09_Norm' 'PGW_06_104_10_Norm'
 'PGW_06_072_02_Norm' 'PGW_06_072_03_Norm' 'PGW_06_072_04_Norm'
 'PGW_06_088_12_Norm' 'PGW_06_088_16_Norm' 'PGW_06_088_18_Norm']
11 calls, function value, improvement, quality: 5029.9, 3.42, 

In [ ]:
r.get_sed?

## Seed finding after stage "tables" 
The "tables" stage triggers a analyze.hptables  run, that executes HPtables.make_seeds(), which in turn runs pipeline.check_ts.make_seeds(). [A little awkward.]
I need to introduce a filter in make_seeds to ignore the path of the Sun for that month.

See [monthly Sun position](monthly Sun position.ipynb), which generated the file \$FERMI/ecliptic\_512.pickle with the ecliptic tuple for each position index in nside=512 (RING as usual). 

Also see the file `\$FERMI/skymodels/P301\_monthly/month\_info.csv`, which has info for each month.

In [ ]:
info = pd.read_csv(open(os.path.expandvars('$FERMI/skymodels/P301_monthly/month_info.csv')),index_col=0)
info.ix[14]

The sun\_ values for for the start of the period. Develop the following function to return a mask. See [Test tables, seed finding](Test tables, seed finding.ipynb)

In [ ]:
import pickle
def monthly_ecliptic_mask( month, elat_max=5):
    """return a nside=512 mask for the given month, an integer starting at 1"""
    info = pd.read_csv(open(os.path.expandvars(
                '$FERMI/skymodels/P301_monthly/month_info.csv')),index_col=0)
    ecliptic_info= pickle.load(open(os.path.expandvars('$FERMI/misc/ecliptic_512.pickle')))
    elon_vec = ecliptic_info[:,0]
    elat_mask = np.abs(ecliptic_info[:,1])<5
    elon_min = info.sun_elon[month]
    try:
        elon_max = info.sun_elon[month+1]
    except:
        elon_max = elon_min+30
    if elon_min< elon_max:
        elon_mask = (elon_vec>elon_min) & (elon_vec<elon_max)
    else:
        elon_mask = (elon_vec>elon_min) | (elon_vec<elon_max)
                                        
    return elat_mask & elon_mask
e_mask=monthly_ecliptic_mask(1); sum(e_mask)

In [ ]:
from uw.like2.pub import healpix_map;
mask_check = healpix_map.HParray('mask_check', e_mask)
mask_check.plot()

In [ ]:
e_mask=monthly_ecliptic_mask(8); sum(e_mask)

In [ ]:
healpix_map.HParray('mask2_check', e_mask).plot()

## Add the TS map seeds

In [ ]:
assert os.path.exists('hptables_ts_kde_512.fits')
assert os.path.exists('seeds_ts.txt')

In [ ]:
from uw.like2 import process; reload(process)
r = process.Process('.', seed_key='ts' ,quiet=True)

In [ ]:
r.setup_roi(174)

In [ ]:
r.summarize()

In [ ]:
from uw.like2 import seeds; reload(seeds)
t =seeds.read_seedfile('ts')
r.process()

In [ ]:
r.print_summary()

In [ ]:
freenames = np.array([ s.name for s  in r.free_sources])
freenames